In [1]:
%matplotlib inline
import os
import sys
sys.path.append("./models/")
sys.path.append("./models/object_detection")

import time

import numpy as np
from matplotlib import pyplot as plt
import cv2
import skimage.io as io
import tensorflow as tf

from utils import label_map_util
from utils import visualization_utils as vis_util

In [2]:
PATH_TO_CKPT = os.path.join('ssd_mobilenet_v1_coco_11_06_2017', 'frozen_inference_graph.pb')
PATH_TO_LABELS = os.path.join('models', 'object_detection', 'data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90

In [3]:
# Load a (frozen) Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [4]:
# Loading label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [5]:
#TEST_IMAGE_PATHS = [ os.path.join('models', 'object_detection', 'test_images', 'image{}.jpg'.format(i)) for i in range(1, 3) ]
TEST_IMAGE_PATHS = ["/var/tmp/_3e11d8d9-fb61-4f25-abaa-b402dad002dc_"+str(1000000+i)[1:]+".jpg" for i in range(1,31)]

In [8]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:

            start = time.time()
            image_np = io.imread(image_path).astype(np.uint8)
            print(image_np.shape)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            print("predict", time.time() - start, "sec")
            
            '''
            start = time.time()
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
            plt.figure(figsize=(12, 12))
            plt.imshow(image_np)
            print("show", time.time() - start, "sec")
            '''

(1024, 3216, 3)
predict 1.2670073509216309 sec
(1024, 3216, 3)
predict 0.4324333667755127 sec
(1024, 3216, 3)
predict 0.41863346099853516 sec
(1024, 3216, 3)
predict 0.43596553802490234 sec
(1024, 3216, 3)
predict 0.4316527843475342 sec
(1024, 3216, 3)
predict 0.4157249927520752 sec
(1024, 3216, 3)
predict 0.41924262046813965 sec
(1024, 3216, 3)
predict 0.4262089729309082 sec
(1024, 3216, 3)
predict 0.46962857246398926 sec
(1024, 3216, 3)
predict 0.42383337020874023 sec
(1024, 3216, 3)
predict 0.41904139518737793 sec
(1024, 3216, 3)
predict 0.4665555953979492 sec
(1024, 3216, 3)
predict 0.4207916259765625 sec
(1024, 3216, 3)
predict 0.4217567443847656 sec
(1024, 3216, 3)
predict 0.4544506072998047 sec
(1024, 3216, 3)
predict 0.49613142013549805 sec
(1024, 3216, 3)
predict 0.4247782230377197 sec
(1024, 3216, 3)
predict 0.472545862197876 sec
(1024, 3216, 3)
predict 0.4213228225708008 sec
(1024, 3216, 3)
predict 0.4507319927215576 sec
(1024, 3216, 3)
predict 0.45179271697998047 sec
(1024,